installing required libraries

In [ ]:
!pip install pandas numpy tensorflow  keras prophet scikit-learn statsmodels

importing libraries

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_percentage_error as mape

change the name to the csv file desired to run the script on 

In [ ]:
datasample = 'sample.xlsx' #<<<<change file name
df = pd.read_excel(datasample)

Displaying the data 

In [ ]:
display(df)

specify the number where the time series starts.

In [ ]:
columns = df.columns

print(f"Column Number -\tColumn Name")
for i in range(len(columns)):
    print(f"{i + 1} -\t\t{columns[i]}")
ts_col_index = int(input("specify the column number where the time series starts. ")) - 1

In [ ]:
ts_col = df.iloc[:, ts_col_index:].columns

preproccessing the data

In [ ]:
remove_outliers = input("Do you want to remove outliers? (yes/no): ").strip().lower() == 'yes'

for feature in ts_col:
    # Calculate the median of the current column
    median_val = df[feature].median()
    
    # Replace zeros with the median value
    df[feature].replace(0, median_val, inplace=True)
    
    # Fill null values with the median value
    df[feature].fillna(median_val, inplace=True)
    
    if remove_outliers:
        # Calculate the interquartile range (IQR) for outlier detection
        first_quartile = df[feature].quantile(0.25)
        third_quartile = df[feature].quantile(0.75)
        iqr = third_quartile - first_quartile
        
        # Define the acceptable range to detect outliers
        lower_bound = first_quartile - 1.5 * iqr
        upper_bound = third_quartile + 1.5 * iqr
        
        # Mark outliers as NaN
        non_outliers = df[feature].between(lower_bound, upper_bound)
        df[feature] = df[feature].where(non_outliers, np.nan)
        
if remove_outliers:
    for feature in ts_col:
        median_val = df[feature].median()
        df[feature].fillna(median_val, inplace=True)

In [ ]:
display(df)

Calculating predictions:

Calculating ARIMA

In [ ]:
def forecast_ARIMA(series, forecast_steps=3):
    arima_model = ARIMA(series, order=(1, 1, 0))
    fitted_model = arima_model.fit()
    future_predictions = fitted_model.forecast(steps=forecast_steps)
    return future_predictions

Calculating Prophet

In [ ]:
def forecast_Prophet(dataframe, forecast_steps=3):
    # Convert the series to a dataframe
    df = pd.DataFrame({'ds': dataframe.index, 'y': dataframe.values})
    prophet_model = Prophet()
    prophet_model.fit(df)
    future_dates = prophet_model.make_future_dataframe(periods=forecast_steps, freq='Y')
    forecast = prophet_model.predict(future_dates)
    return forecast['yhat'][-forecast_steps:].values

Calculating LSTM

In [ ]:
def forecast_LSTM(series, forecast_steps=3):
    # Reshape the series for scaling
    series = series.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    series_scaled = scaler.fit_transform(series)
    
    # Prepare the dataset for LSTM
    X, y = [], []
    for i in range(len(series_scaled) - forecast_steps):
        X.append(series_scaled[i:i+forecast_steps, 0])
        y.append(series_scaled[i+1:i+forecast_steps+1, 0])  
    
    X = np.array(X)
    y = np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(forecast_steps, 1)))
    model.add(LSTM(50))
    model.add(Dense(forecast_steps))
    optimizer = Adam(learning_rate=0.001, clipvalue=1.0)  
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Fit the model
    model.fit(X, y, epochs=10, batch_size=32, verbose=0)
    
    # Make predictions
    last_sequence = series_scaled[-forecast_steps:].reshape((1, forecast_steps, 1))
    predicted_scaled = model.predict(last_sequence)
    predictions = scaler.inverse_transform(predicted_scaled)
    
    # Return final LSTM predictions
    return predictions.flatten()


Writing the prediction into a dataframe

In [12]:
# Iterate through each row for forecasting
results = []
for index, row in df.iterrows():
    series = row.iloc[ts_col_index:]
    actual_values = series[-3:].values
    series = series.apply(pd.to_numeric, errors='coerce')
    # ARIMA Predictions and MAPE
    arima_predictions = forecast_ARIMA(series[:-3])
    arima_mape = [round(mape([actual_values[i]], [arima_predictions[i]]), 2) * 100 for i in range(len(actual_values))]
    # Prophet Predictions and MAPE
    prophet_series = row.iloc[ts_col_index-1:]
    prophet_predictions = forecast_Prophet(series)
    prophet_mape = [round(mape([actual_values[i]], [prophet_predictions[i]]), 2) * 100 for i in range(len(actual_values))]
    # LSTM Predictions and MAPE
    lstm_predictions = forecast_LSTM(series)
    lstm_mape = [round(mape([actual_values[i]], [lstm_predictions[i]]), 2) * 100 for i in range(len(actual_values))]
    # Extract additional information dynamically
    explanatory_data = {}
    for col_name in row.index:
        if col_name not in ts_col:
            explanatory_data[col_name] = row[col_name]
    results.append({
        'ARIMA_Pred1': round(arima_predictions[0], 2), 
        'ARIMA_Pred2': round(arima_predictions[1], 2),
        'ARIMA_Pred3': round(arima_predictions[2], 2),
        'ARIMA_Pred1_MAPE': round(arima_mape[0], 2), 
        'ARIMA_Pred2_MAPE': round(arima_mape[1], 2),
        'ARIMA_Pred3_MAPE': round(arima_mape[2], 2),
        'Prophet_Pred1': round(prophet_predictions[0], 2),
        'Prophet_Pred2': round(prophet_predictions[1], 2),
        'Prophet_Pred3': round(prophet_predictions[2], 2),
        'Prophet_Pred1_MAPE': round(prophet_mape[0], 2), 
        'Prophet_Pred2_MAPE': round(prophet_mape[1], 2),
        'Prophet_Pred3_MAPE': round(prophet_mape[2], 2),
        'LSTM_Pred1': round(lstm_predictions[0], 2), 
        'LSTM_Pred2': round(lstm_predictions[1], 2),
        'LSTM_Pred3': round(lstm_predictions[2], 2),
        'LSTM_Pred1_MAPE': round(lstm_mape[0], 2), 
        'LSTM_Pred2_MAPE': round(lstm_mape[1], 2),
        'LSTM_Pred3_MAPE': round(lstm_mape[2], 2),
    })

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step


C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred1': round(arima_predictions[0], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred2': round(arima_predictions[1], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step


C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred1': round(arima_predictions[0], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred2': round(arima_predictions[1], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step


...

C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred1': round(arima_predictions[0], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred2': round(arima_predictions[1], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step


C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred1': round(arima_predictions[0], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred2': round(arima_predictions[1], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step


C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred1': round(arima_predictions[0], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ARIMA_Pred2': round(arima_predictions[1], 2),
C:\Users\altme\AppData\Local\Temp\ipykernel_18148\2819226725.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

exporting the final output as csv file

In [13]:
# Create DataFrame from results
results_df = pd.DataFrame(results)

# Reset index of both DataFrames
df.reset_index(drop=True, inplace=True)
results_df.reset_index(drop=True, inplace=True)

# Merge with original data
merged_df = pd.concat([df, results_df], axis=1)

# Save the final output to a new CSV file
merged_df.to_csv('final_output.csv', index=False)